In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

/home/t/tianqi/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_transforms = transforms.Compose([
                                 transforms.ToTensor()
])

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms)
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
torch.cuda.get_device_properties(torch.device)

_CudaDeviceProperties(name='NVIDIA TITAN RTX', major=7, minor=5, total_memory=24220MB, multi_processor_count=72)

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 3)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/home/t/tianqi/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/t/tianqi/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 1.1136 Acc: 0.4774
val Loss: 0.8795 Acc: 0.6772
test Loss: 0.9526 Acc: 0.6316

Epoch 1/24
----------
train Loss: 0.8516 Acc: 0.6418
val Loss: 0.7527 Acc: 0.6798
test Loss: 0.7312 Acc: 0.7105

Epoch 2/24
----------
train Loss: 0.6715 Acc: 0.7255
val Loss: 0.6791 Acc: 0.7428
test Loss: 0.5716 Acc: 0.7789

Epoch 3/24
----------
train Loss: 0.5863 Acc: 0.7745
val Loss: 0.6060 Acc: 0.7507
test Loss: 0.6072 Acc: 0.7579

Epoch 4/24
----------
train Loss: 0.4552 Acc: 0.8145
val Loss: 0.5249 Acc: 0.7874
test Loss: 0.5193 Acc: 0.7842

Epoch 5/24
----------
train Loss: 0.4111 Acc: 0.8394
val Loss: 0.7095 Acc: 0.7795
test Loss: 0.6402 Acc: 0.7737

Epoch 6/24
----------
train Loss: 0.4095 Acc: 0.8439
val Loss: 0.8683 Acc: 0.7218
test Loss: 0.9432 Acc: 0.7053

Epoch 7/24
----------
train Loss: 0.2425 Acc: 0.9087
val Loss: 0.4481 Acc: 0.8373
test Loss: 0.3903 Acc: 0.8316

Epoch 8/24
----------
train Loss: 0.1922 Acc: 0.9382
val Loss: 0.4263 Acc: 0.8530
test Loss: 0.3

In [10]:
torch.save(model_ft.state_dict(), "resnet50.pt")

In [23]:
from sklearn.metrics import classification_report

y_pred = []
y_true = []

model_ft.eval()

for inputs, labels in dataloaders["test"]:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model_ft(inputs)
    _, preds = torch.max(outputs, 1)
    y_pred.extend(preds.data.cpu())
    y_true.extend(labels.data.cpu())


print(classification_report(y_true, y_pred, labels=[0,1,2]))

              precision    recall  f1-score   support

           0       0.76      0.95      0.84        79
           1       0.94      0.72      0.82        43
           2       0.95      0.81      0.87        68

    accuracy                           0.85       190
   macro avg       0.88      0.83      0.84       190
weighted avg       0.87      0.85      0.85       190

